In [10]:
import cv2
import numpy as np
import urllib

hostname = "10.1.91.167" # IP-Adresse des Roboters
i = 0 # Laufnummer, für mehrere Bilder hintereinander

In [2]:
# Bild aus Website auslesen als statisches Bild und abspeichern
resp = urllib.request.urlopen(f"http://{hostname}:4242/current.jpg?type=color")
image = np.asarray(bytearray(resp.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_COLOR)

# Bild speicher (Pfad anpassen)
cv2.imwrite(f"images/image_{i}.jpg", image)
i += 1


In [3]:
# Bild aus Website einlesen als Video und gegebenfalls speichern
i = 0
while True:
    # download the image, convert it to a NumPy array, and then show it
    resp = urllib.request.urlopen(f"http://{hostname}:4242/current.jpg?type=color")
    image_array = np.asarray(bytearray(resp.read()), dtype="uint8")
    img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    height, widgth, _ = img.shape
    #cv2.rectangle(img, (int(widgth / 2 - 25), height - 100), (int(widgth / 2 + 25), height - 50), (255,0,0), 2)
    cv2.imshow("Cam", img)

    # Tastatur befehle ermitteln und ausgeben
    key = cv2.waitKey(1)
    if key == ord('r'): # Bild aufnehmen
        cv2.imwrite(f"images/image_{i}.jpg", img)
        i += 1   
    elif key == ord('q'): # Video beenden 
        break

cv2.destroyAllWindows()

In [11]:
# Bild aus Website einlesen als Video anzeigen und Buchstaben ermitteln
# Taste q - Beenden
# Taste r - Bild speichern
# Taste s - Durchschnitts HSV-Werte ausgeben
while True:
    # download the image, convert it to a OpenCV numpy arry (BGR)
    resp = urllib.request.urlopen(f"http://{hostname}:4242/current.jpg?type=color")
    image_array = np.asarray(bytearray(resp.read()), dtype="uint8")
    img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    # get shape and 
    height, widgth, _ = img.shape
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Bildbereich welcher ausgewertet werden soll
    height, widgth, _ = img.shape
    squareToCheck = hsv[height - 95: height - 55, int(widgth / 2 - 20): int(widgth / 2 + 20), :]

    # Durschschnitts HSV-Werte des Bildbereichs ermitteln
    sum = [0, 0, 0]
    posibility = 0
    numberOfPixelToCheck = 0
    for line in squareToCheck:
        for pixel in line:
            sum[0] += pixel[0]
            sum[1] += pixel[1]
            sum[2] += pixel[2]
            numberOfPixelToCheck += 1

            if pixel[0] < 10 or pixel[1] > 330:
                posibility += 1

    average = [sum[0] / numberOfPixelToCheck, sum[1] / numberOfPixelToCheck, sum[2] / numberOfPixelToCheck]

    # Funktion zum umnormieren des Durchschnittwertes (H - 360, S - 100, V - 100)
    def normalize(value, value_max, norm_max):
        return value * norm_max / value_max

    # Anhand der Durchschnittswerte, Farbe (Buchstabe) erkennen und Ergebnis ins Bild schreiben
    if posibility / numberOfPixelToCheck > 0.75:
        img = cv2.putText(img, 'I', (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0 , 255), 2, cv2.LINE_AA)
    elif normalize(60, 360, 255) <= average[0] <= normalize(140, 360, 255):
        if normalize(0, 100, 255) <= average[1] <= normalize(50, 100, 255):
            img = cv2.putText(img, 'A', (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0 , 255), 2, cv2.LINE_AA)
        elif normalize(50, 100, 255) <= average[1] <= normalize(100, 100, 255):
            img = cv2.putText(img, 'F', (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0 , 255), 2, cv2.LINE_AA)
    elif normalize(22, 360, 255) <=  average[0] <= normalize(55, 360, 255):
        if normalize(0, 100, 255) <= average[1] <= normalize(50, 100, 255):
            img = cv2.putText(img, 'U', (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0 , 255), 2, cv2.LINE_AA)
        elif normalize(50, 100, 255) <= average[1] <= normalize(100, 100, 255):
            img = cv2.putText(img, 'O', (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0 , 255), 2, cv2.LINE_AA)
    elif normalize(150, 360, 255) <=  average[0] <= normalize(240, 360, 255):
        img = cv2.putText(img, 'R', (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0 , 255), 2, cv2.LINE_AA)

    # Bild anzeigen
    cv2.imshow("Cam", img)

    # Tastaturbefehle ermitteln und ausführen
    key = cv2.waitKey(1)
    if key == ord('r'): # Bild aufnehmen
        cv2.imwrite(f"images/image_{i}.jpg", img)
        i += 1
    elif key == ord('q'): # Video beenden
        break
    elif key == ord('s'): # Durchschnittswerte ausgeben
        average[0] = normalize(average[0], 255, 360)
        average[1] = normalize(average[1], 255, 100)
        average[2] = normalize(average[2], 255, 100)
        print(average)
    
cv2.destroyAllWindows()
